Step 1: Setup and Environment
First, create a new Jupyter notebook and add a markdown cell with your project title and description.

Then, add a code cell for imports:

In [ ]:
!python --version
!pip install umap-learn scanpy scipy anndata leidenalg python-igraph louvain


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
from scipy import stats

# SC analysis
import scanpy as sc
import anndata as ad

# dr methods
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap

# evaluation metrics
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.neighbors import NearestNeighbors

# random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# talked about the configuration in the report
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=100, facecolor='white', frameon=True)

try:
    sc.logging.print_header()
except Exception as e:
    print(f"Note: Could not print scanpy header: {str(e)}")

Step 2: Data Acquisition
Add a code cell to download and load datasets:

In [ ]:
def get_datasets(download_path="data"):
    """Download and prepare datasets for analysis"""
    
    os.makedirs(download_path, exist_ok=True)
    
    # PBMC
    try:
        pbmc = sc.read_h5ad(os.path.join(download_path, "pbmc3k.h5ad"))
        print("PBMC dataset loaded from file.")
    except FileNotFoundError:
        print("Downloading PBMC dataset...")
        pbmc = sc.datasets.pbmc3k()
        pbmc.write_h5ad(os.path.join(download_path, "pbmc3k.h5ad"))
    
    # Developmental trajectory
    try:
        trajectory = sc.read_h5ad(os.path.join(download_path, "paul15.h5ad"))
        print("Paul et al. trajectory dataset loaded from file.")
    except FileNotFoundError:
        print("Downloading Paul et al. trajectory dataset...")
        trajectory = sc.datasets.paul15()
        trajectory.write_h5ad(os.path.join(download_path, "paul15.h5ad"))
    
    return pbmc, trajectory


pbmc_data, trajectory_data = get_datasets()

# info about datasets
print("\nPBMC dataset summary:")
print(f"Shape: {pbmc_data.shape}")
print(f"Available annotations: {list(pbmc_data.obs.columns)}")

print("\nTrajectory dataset summary:")
print(f"Shape: {trajectory_data.shape}")
print(f"Available annotations: {list(trajectory_data.obs.columns)}")

Step 3: Data Preprocessing Function
Add a preprocessing function in a new code cell:

In [ ]:

def preprocess_data(adata, min_genes=200, min_cells=3, 
                   target_sum=1e4, n_top_genes=5000,
                   quick_run=False):
    """
    Preprocess AnnData object for dimensionality reduction
    
    Parameters:
    -----------
    adata : AnnData
        AnnData object to preprocess
    min_genes : int
        Minimum number of genes per cell
    min_cells : int
        Minimum number of cells per gene
    target_sum : float
        Target sum for normalization
    n_top_genes : int
        Number of highly variable genes to select
    quick_run : bool
        If True, subset to 1000 cells for quicker testing
    
    Returns:
    --------
    AnnData
        Preprocessed data
    """
    print(f"Starting preprocessing with {adata.shape[0]} cells and {adata.shape[1]} genes")
    
    adata = adata.copy()
    
    if quick_run and adata.shape[0] > 1000:
        print("Subsampling to 1000 cells for quick run")
        sc.pp.subsample(adata, n_obs=1000, random_state=RANDOM_SEED)
    
    # QC filtering
    sc.pp.filter_cells(adata, min_genes=min_genes)
    sc.pp.filter_genes(adata, min_cells=min_cells)
    
    # QC metrics
    sc.pp.calculate_qc_metrics(adata, inplace=True)

    # mitochondrial gene percentage
    mito_prefixes = ['MT-', 'mt-', 'Mt-']
    has_mito = False

    for prefix in mito_prefixes:
        if any(adata.var_names.str.startswith(prefix)):
            adata.var['mt'] = adata.var_names.str.startswith(prefix)
            has_mito = True
            print(f"Found mitochondrial genes with prefix: {prefix}")
            break

    if not has_mito:
        print("No mitochondrial genes found with standard prefixes")
        adata.var['mt'] = False
    
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, inplace=True)
    
    adata.obs['pct_counts_mt'] = adata.obs['pct_counts_mt']
    
    # normalizaion - (Not Poisson/NB, but I talked about this in the report)
    sc.pp.normalize_total(adata, target_sum=target_sum)
    sc.pp.log1p(adata)
    
    # hvg
    sc.pp.highly_variable_genes(adata, n_top_genes=n_top_genes)
    
    # subset to hvg
    adata = adata[:, adata.var.highly_variable]
    
    # scaleing
    sc.pp.scale(adata, max_value=10)
    
    print(f"Finished preprocessing: {adata.shape[0]} cells and {adata.shape[1]} genes remain")
    return adata

Step 4: Preprocessing the Data


In [ ]:
QUICK_RUN = False

pbmc_processed = preprocess_data(pbmc_data, quick_run=QUICK_RUN)
trajectory_processed = preprocess_data(trajectory_data, quick_run=QUICK_RUN)

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# PBMC QC plots
sns.violinplot(y=pbmc_processed.obs['n_genes'], ax=axes[0, 0])
sns.violinplot(y=pbmc_processed.obs['total_counts'], ax=axes[0, 1])
sns.violinplot(y=pbmc_processed.obs['pct_counts_mt'], ax=axes[0, 2])
axes[0, 0].set_title('PBMC: Genes per Cell')
axes[0, 1].set_title('PBMC: Counts per Cell')
axes[0, 2].set_title('PBMC: % Mitochondrial')

# trajectory QC plots
sns.violinplot(y=trajectory_processed.obs['n_genes'], ax=axes[1, 0])
sns.violinplot(y=trajectory_processed.obs['total_counts'], ax=axes[1, 1])
axes[1, 0].set_title('Trajectory: Genes per Cell')
axes[1, 1].set_title('Trajectory: Counts per Cell')
axes[1, 2].set_visible(False)

plt.tight_layout()
plt.show()

Step 5: Dimensionality Reduction Functions
Add a cell with dimensionality reduction functions:

In [ ]:

def run_pca(adata, n_comps=[10, 20, 50, 100], use_hvg=True):
    results = {}
    
    data = adata.copy()
    
    for n_comp in n_comps:
        start_time = time.perf_counter()
        
        # limit to max possible components
        actual_n_comp = min(n_comp, data.shape[0] - 1, data.shape[1])
        
        print(f"Running PCA with {actual_n_comp} components...")
        sc.pp.pca(data, n_comps=actual_n_comp, use_highly_variable=use_hvg, random_state=RANDOM_SEED)
        
        end_time = time.perf_counter()
        runtime = end_time - start_time
        
        results[n_comp] = {
            'embedding': data.obsm['X_pca'].copy(),
            'runtime': runtime,
            'variance_ratio': data.uns['pca']['variance_ratio']
        }
        
        print(f"PCA with {actual_n_comp} components completed in {runtime:.2f} seconds")
        
    return results

def run_tsne(adata, perplexities=[5, 30, 50], n_pca=50, random_seeds=[42, 123, 456]):
    results = {}
    
    data = adata.copy()
    if 'X_pca' not in data.obsm:
        sc.pp.pca(data, n_comps=n_pca, random_state=RANDOM_SEED)
    
    for perplexity in perplexities:
        results[perplexity] = {'embeddings': [], 'runtimes': []}
        
        for seed in random_seeds:
            print(f"Running t-SNE with perplexity {perplexity}, seed {seed}...")
            
            start_time = time.perf_counter()
            
            sc.tl.tsne(data, perplexity=perplexity, use_rep='X_pca', 
                       random_state=seed, n_jobs=4)
            
            end_time = time.perf_counter()
            runtime = end_time - start_time
            
            results[perplexity]['embeddings'].append(data.obsm['X_tsne'].copy())
            results[perplexity]['runtimes'].append(runtime)
            
            print(f"t-SNE with perplexity {perplexity} completed in {runtime:.2f} seconds")
    
    return results

def run_umap(adata, n_neighbors_list=[5, 15, 30, 50], min_dist_list=[0.1, 0.5], n_pca=50):
    results = {}
    
    data = adata.copy()
    if 'X_pca' not in data.obsm:
        sc.pp.pca(data, n_comps=n_pca, random_state=RANDOM_SEED)
    
    for n_neighbors in n_neighbors_list:
        results[n_neighbors] = {}
        
        for min_dist in min_dist_list:
            print(f"Running UMAP with n_neighbors={n_neighbors}, min_dist={min_dist}...")
            
            start_time = time.perf_counter()
            
            sc.pp.neighbors(data, n_neighbors=n_neighbors, use_rep='X_pca', random_state=RANDOM_SEED)
            sc.tl.umap(data, min_dist=min_dist, random_state=RANDOM_SEED)
            
            end_time = time.perf_counter()
            runtime = end_time - start_time
            
            results[n_neighbors][min_dist] = {
                'embedding': data.obsm['X_umap'].copy(),
                'runtime': runtime
            }
            
            print(f"UMAP with n_neighbors={n_neighbors}, min_dist={min_dist} completed in {runtime:.2f} seconds")
    
    return results

Step 6: Running the Dimensionality Reduction Methods
Add a cell to run the methods:

In [ ]:

print("Running dimensionality reduction on PBMC data...")

pbmc_pca_results = run_pca(pbmc_processed, n_comps=[10, 20, 50, 100])
pbmc_tsne_results = run_tsne(pbmc_processed, perplexities=[5, 30, 50])
pbmc_umap_results = run_umap(pbmc_processed, n_neighbors_list=[5, 15, 30, 50], min_dist_list=[0.1, 0.5])

print("\nRunning dimensionality reduction on trajectory data...")

trajectory_pca_results = run_pca(trajectory_processed, n_comps=[10, 20, 50, 100])
trajectory_tsne_results = run_tsne(trajectory_processed, perplexities=[5, 30, 50])
trajectory_umap_results = run_umap(trajectory_processed, n_neighbors_list=[5, 15, 30, 50], min_dist_list=[0.1, 0.5])

print("All dimensionality reduction methods completed!")

Step 7: Evaluation Metrics
Add a cell with evaluation metric functions:

In [ ]:

def evaluate_clustering_preservation(embedding, labels):
    """
    # how well clustering is preserved 
    # leiden clustering
    
    Returns ARI and NMI scores
    """
    temp = ad.AnnData(X=np.zeros((embedding.shape[0], 1)))
    temp.obsm['X_embedding'] = embedding
    
    sc.pp.neighbors(temp, use_rep='X_embedding')
    
    resolutions = [0.1, 0.3, 0.5, 0.8, 1.0, 1.5, 2.0]
    best_ari = -1
    best_nmi = -1
    
    for res in resolutions:
        sc.tl.leiden(temp, resolution=res)
        pred_labels = temp.obs['leiden'].astype(str).values
        
        ari = adjusted_rand_score(labels, pred_labels)
        nmi = normalized_mutual_info_score(labels, pred_labels)
        
        if ari > best_ari:
            best_ari = ari
            
        if nmi > best_nmi:
            best_nmi = nmi
    
    return best_ari, best_nmi

def calculate_knn_preservation(high_dim, low_dim, k=15):
    """
    this, calculates KNN preservation between high-dimensional and low-dimensional spaces
    """
    # find knn in high-dimensional space
    high_nbrs = NearestNeighbors(n_neighbors=k+1).fit(high_dim)
    high_indices = high_nbrs.kneighbors(high_dim, return_distance=False)
    
    # find knn in low-dimensional space
    low_nbrs = NearestNeighbors(n_neighbors=k+1).fit(low_dim)
    low_indices = low_nbrs.kneighbors(low_dim, return_distance=False)
    
    # calculate overlap
    overlap_sum = 0
    for i in range(high_dim.shape[0]):
        high_set = set(high_indices[i, 1:])
        low_set = set(low_indices[i, 1:])
        overlap_sum += len(high_set.intersection(low_set))
    
    # preservation ratio
    preservation = overlap_sum / (high_dim.shape[0] * k)
    return preservation

def evaluate_embeddings(adata, dr_results, method_name, cell_type_key):    
    high_dim = adata.X
    
    if cell_type_key in adata.obs:
        labels = adata.obs[cell_type_key].astype(str).values
    else:
        print(f"Warning: {cell_type_key} not found in data. Skipping clustering metrics.")
        labels = None
    
    results = {}
    
    if method_name == "pca":
        for n_comp, data in dr_results.items():
            embedding = data['embedding']
            runtime = data['runtime']
            
            results[n_comp] = {'runtime': runtime}
            
            if labels is not None:
                ari, nmi = evaluate_clustering_preservation(embedding, labels)
                results[n_comp]['ari'] = ari
                results[n_comp]['nmi'] = nmi
            
            results[n_comp]['knn_preservation'] = calculate_knn_preservation(high_dim, embedding)
    
    elif method_name == "tsne":
        for perplexity, data in dr_results.items():
            avg_runtime = np.mean(data['runtimes'])
            results[perplexity] = {'runtime': avg_runtime}
            
            # average metrics across random seeds
            ari_values = []
            nmi_values = []
            knn_values = []
            
            for i, embedding in enumerate(data['embeddings']):
                if labels is not None:
                    ari, nmi = evaluate_clustering_preservation(embedding, labels)
                    ari_values.append(ari)
                    nmi_values.append(nmi)
                
                knn_values.append(calculate_knn_preservation(high_dim, embedding))
            
            if labels is not None:
                results[perplexity]['ari'] = np.mean(ari_values)
                results[perplexity]['nmi'] = np.mean(nmi_values)
            
            results[perplexity]['knn_preservation'] = np.mean(knn_values)
    
    elif method_name == "umap":
        for n_neighbors, min_dist_data in dr_results.items():
            results[n_neighbors] = {}
            
            for min_dist, data in min_dist_data.items():
                embedding = data['embedding']
                runtime = data['runtime']
                
                results[n_neighbors][min_dist] = {'runtime': runtime}
                
                if labels is not None:
                    ari, nmi = evaluate_clustering_preservation(embedding, labels)
                    results[n_neighbors][min_dist]['ari'] = ari
                    results[n_neighbors][min_dist]['nmi'] = nmi
                
                results[n_neighbors][min_dist]['knn_preservation'] = calculate_knn_preservation(high_dim, embedding)
    
    return results

In [ ]:
print("Computing clusters for PBMC dataset...")

if 'X_pca' not in pbmc_processed.obsm:
    sc.pp.pca(pbmc_processed, n_comps=40, random_state=RANDOM_SEED)

# knn on pca
sc.pp.neighbors(pbmc_processed, n_neighbors=10, use_rep='X_pca')

# Louvain
sc.tl.louvain(pbmc_processed, resolution=0.5, random_state=RANDOM_SEED)
print("PBMC Louvain clusters:", pbmc_processed.obs['louvain'].unique())

# Leiden
sc.tl.leiden(pbmc_processed, resolution=0.5, random_state=RANDOM_SEED)
print("PBMC Leiden clusters:", pbmc_processed.obs['leiden'].unique())

# Louvain vs Leiden
ari_pbmc = adjusted_rand_score(
    pbmc_processed.obs['louvain'].values,
    pbmc_processed.obs['leiden'].values
)
print(f"PBMC Louvain vs Leiden ARI: {ari_pbmc:.4f}")

print("\nComputing clusters for trajectory dataset...")

if 'X_pca' not in trajectory_processed.obsm:
    sc.pp.pca(trajectory_processed, n_comps=40, random_state=RANDOM_SEED)

# knn on pca
sc.pp.neighbors(trajectory_processed, n_neighbors=10, use_rep='X_pca')

# Louvain
if 'paul15_clusters' not in trajectory_processed.obs:
    sc.tl.louvain(trajectory_processed, resolution=0.5, key_added='paul15_clusters', random_state=RANDOM_SEED)

# Leiden
sc.tl.leiden(trajectory_processed, resolution=0.5, key_added='paul15_clusters_leiden', random_state=RANDOM_SEED)

# Louvain vs Leiden
ari_trajectory = adjusted_rand_score(
    trajectory_processed.obs['paul15_clusters'].values,
    trajectory_processed.obs['paul15_clusters_leiden'].values
)
print(f"Trajectory Louvain vs Leiden ARI: {ari_trajectory:.4f}")

Step 8: Running the Evaluations

In [ ]:
pbmc_louvain_key = 'louvain'  
pbmc_leiden_key = 'leiden'
trajectory_louvain_key = 'paul15_clusters'
trajectory_leiden_key = 'paul15_clusters_leiden'

# PBMC with Louvain
print("Evaluating PBMC dimensionality reduction results with Louvain clusters...")
pbmc_pca_eval_louvain = evaluate_embeddings(pbmc_processed, pbmc_pca_results, "pca", pbmc_louvain_key)
pbmc_tsne_eval_louvain = evaluate_embeddings(pbmc_processed, pbmc_tsne_results, "tsne", pbmc_louvain_key)
pbmc_umap_eval_louvain = evaluate_embeddings(pbmc_processed, pbmc_umap_results, "umap", pbmc_louvain_key)

# PBMC with Leiden
print("Evaluating PBMC dimensionality reduction results with Leiden clusters...")
pbmc_pca_eval_leiden = evaluate_embeddings(pbmc_processed, pbmc_pca_results, "pca", pbmc_leiden_key)
pbmc_tsne_eval_leiden = evaluate_embeddings(pbmc_processed, pbmc_tsne_results, "tsne", pbmc_leiden_key)
pbmc_umap_eval_leiden = evaluate_embeddings(pbmc_processed, pbmc_umap_results, "umap", pbmc_leiden_key)

# trajectory with Louvain
print("\nEvaluating trajectory dimensionality reduction results with Louvain clusters...")
traj_pca_eval_louvain = evaluate_embeddings(trajectory_processed, trajectory_pca_results, "pca", trajectory_louvain_key)
traj_tsne_eval_louvain = evaluate_embeddings(trajectory_processed, trajectory_tsne_results, "tsne", trajectory_louvain_key)
traj_umap_eval_louvain = evaluate_embeddings(trajectory_processed, trajectory_umap_results, "umap", trajectory_louvain_key)

# trajectory with Leiden
print("\nEvaluating trajectory dimensionality reduction results with Leiden clusters...")
traj_pca_eval_leiden = evaluate_embeddings(trajectory_processed, trajectory_pca_results, "pca", trajectory_leiden_key)
traj_tsne_eval_leiden = evaluate_embeddings(trajectory_processed, trajectory_tsne_results, "tsne", trajectory_leiden_key)
traj_umap_eval_leiden = evaluate_embeddings(trajectory_processed, trajectory_umap_results, "umap", trajectory_leiden_key)

print("All evaluations completed!")

In [ ]:
# # Compute clusters for both datasets
# print("Computing clusters for PBMC dataset...")
# sc.pp.neighbors(pbmc_processed, n_neighbors=10, n_pcs=40)
# sc.tl.louvain(pbmc_processed, resolution=0.5)
# print("PBMC cell types:", pbmc_processed.obs['louvain'].unique())

# print("\nComputing clusters for trajectory dataset...")
# sc.pp.neighbors(trajectory_processed, n_neighbors=10, n_pcs=40)
# if 'paul15_clusters' not in trajectory_processed.obs:
#     # Create clusters if the ground truth isn't available
#     sc.tl.louvain(trajectory_processed, resolution=0.5, key_added='paul15_clusters')

# # Check available columns for reference
# print("\nAvailable columns in PBMC data:")
# print(list(pbmc_processed.obs.columns))
# print("\nAvailable columns in trajectory data:")
# print(list(trajectory_processed.obs.columns))

Step 9: Visualize Results

In [ ]:
def plot_pca_results(adata, pca_results, cell_type_key, title_prefix=""):
    n_plots = len(pca_results)
    fig, axes = plt.subplots(2, n_plots, figsize=(5*n_plots, 10))
    
    for i, (n_comp, result) in enumerate(pca_results.items()):
        # variance plot
        variance_ratio = result['variance_ratio']
        cumulative_variance = np.cumsum(variance_ratio)
        axes[0, i].plot(range(1, len(variance_ratio)+1), cumulative_variance, 'o-')
        axes[0, i].axhline(y=0.8, color='r', linestyle='--', alpha=0.5)
        axes[0, i].set_xlabel('Number of components')
        axes[0, i].set_ylabel('Cumulative explained variance')
        axes[0, i].set_title(f'{title_prefix} PCA n_comp={n_comp}')
        
        # scatter plot
        embedding = result['embedding']
        temp = ad.AnnData(X=adata.X)
        temp.obs = adata.obs
        temp.obsm['X_pca'] = embedding
        
        sc.pl.pca(temp, color=cell_type_key, ax=axes[1, i], show=False)
        axes[1, i].set_title(f'Runtime: {result["runtime"]:.2f} sec')
    
    plt.tight_layout()
    return fig

def plot_tsne_results(adata, tsne_results, cell_type_key, title_prefix=""):
    n_plots = len(tsne_results)
    fig, axes = plt.subplots(1, n_plots, figsize=(5*n_plots, 5))
    
    for i, (perplexity, result) in enumerate(tsne_results.items()):
        # first seed for plotting
        embedding = result['embeddings'][0]
        runtime = result['runtimes'][0]
        
        temp = ad.AnnData(X=adata.X)
        temp.obs = adata.obs
        temp.obsm['X_tsne'] = embedding
        
        sc.pl.tsne(temp, color=cell_type_key, ax=axes[i], show=False)
        axes[i].set_title(f'{title_prefix} t-SNE perp={perplexity}\nRuntime: {runtime:.2f} sec')
    
    plt.tight_layout()
    return fig

def plot_umap_results(adata, umap_results, cell_type_key, title_prefix=""):
    n_neighbors_list = list(umap_results.keys())
    min_dist_list = list(umap_results[n_neighbors_list[0]].keys())
    
    fig, axes = plt.subplots(len(n_neighbors_list), len(min_dist_list), 
                             figsize=(5*len(min_dist_list), 5*len(n_neighbors_list)))
    
    for i, n_neighbors in enumerate(n_neighbors_list):
        for j, min_dist in enumerate(min_dist_list):
            result = umap_results[n_neighbors][min_dist]
            embedding = result['embedding']
            runtime = result['runtime']
            
            temp = ad.AnnData(X=adata.X)
            temp.obs = adata.obs
            temp.obsm['X_umap'] = embedding
            
            ax = axes[i, j] if len(n_neighbors_list) > 1 else axes[j]
            sc.pl.umap(temp, color=cell_type_key, ax=ax, show=False)
            ax.set_title(f'{title_prefix} UMAP n_neigh={n_neighbors}, min_d={min_dist}\nRuntime: {runtime:.2f} sec')
    
    plt.tight_layout()
    return fig


# we use Leiden since it is generally considered better
pbmc_cell_type_key = pbmc_leiden_key
trajectory_cell_type_key = trajectory_leiden_key

# PBMC
plot_pca_results(pbmc_processed, pbmc_pca_results, pbmc_cell_type_key, "PBMC")
plot_tsne_results(pbmc_processed, pbmc_tsne_results, pbmc_cell_type_key, "PBMC")
plot_umap_results(pbmc_processed, pbmc_umap_results, pbmc_cell_type_key, "PBMC")

# trajectory
plot_pca_results(trajectory_processed, trajectory_pca_results, trajectory_cell_type_key, "Trajectory")
plot_tsne_results(trajectory_processed, trajectory_tsne_results, trajectory_cell_type_key, "Trajectory")
plot_umap_results(trajectory_processed, trajectory_umap_results, trajectory_cell_type_key, "Trajectory")

Step 10: Compare Performance Metrics

In [ ]:
def create_performance_summary_with_clustering_comparison(
    # Louvain evaluations
    pbmc_pca_eval_louvain, pbmc_tsne_eval_louvain, pbmc_umap_eval_louvain,
    traj_pca_eval_louvain, traj_tsne_eval_louvain, traj_umap_eval_louvain,
    # Leiden evaluations
    pbmc_pca_eval_leiden, pbmc_tsne_eval_leiden, pbmc_umap_eval_leiden,
    traj_pca_eval_leiden, traj_tsne_eval_leiden, traj_umap_eval_leiden
):
    """summary of performance metrics for all methods and clustering algorithms"""
    
    def extract_metrics(eval_dict, method, dataset, clustering_algorithm):
        metrics_list = []
        
        if method == "PCA":
            for n_comp, metrics in eval_dict.items():
                metrics_list.append({
                    'Method': method,
                    'Dataset': dataset,
                    'Parameters': f'n_comp={n_comp}',
                    'Clustering_Algorithm': clustering_algorithm,
                    'ARI': metrics.get('ari', np.nan),
                    'NMI': metrics.get('nmi', np.nan),
                    'KNN_Preservation': metrics.get('knn_preservation', np.nan),
                    'Runtime_sec': metrics['runtime']
                })
        
        elif method == "t-SNE":
            for perp, metrics in eval_dict.items():
                metrics_list.append({
                    'Method': method,
                    'Dataset': dataset,
                    'Parameters': f'perplexity={perp}',
                    'Clustering_Algorithm': clustering_algorithm,
                    'ARI': metrics.get('ari', np.nan),
                    'NMI': metrics.get('nmi', np.nan),
                    'KNN_Preservation': metrics.get('knn_preservation', np.nan),
                    'Runtime_sec': metrics['runtime']
                })
        
        elif method == "UMAP":
            for n_neigh, min_dist_data in eval_dict.items():
                for min_dist, metrics in min_dist_data.items():
                    metrics_list.append({
                        'Method': method,
                        'Dataset': dataset,
                        'Parameters': f'n_neighbors={n_neigh}, min_dist={min_dist}',
                        'Clustering_Algorithm': clustering_algorithm,
                        'ARI': metrics.get('ari', np.nan),
                        'NMI': metrics.get('nmi', np.nan),
                        'KNN_Preservation': metrics.get('knn_preservation', np.nan),
                        'Runtime_sec': metrics['runtime']
                    })
                    
        return metrics_list
    
    # metrics for Louvain
    pbmc_pca_metrics_louvain = extract_metrics(pbmc_pca_eval_louvain, "PCA", "PBMC", "Louvain")
    pbmc_tsne_metrics_louvain = extract_metrics(pbmc_tsne_eval_louvain, "t-SNE", "PBMC", "Louvain")
    pbmc_umap_metrics_louvain = extract_metrics(pbmc_umap_eval_louvain, "UMAP", "PBMC", "Louvain")
    
    traj_pca_metrics_louvain = extract_metrics(traj_pca_eval_louvain, "PCA", "Trajectory", "Louvain")
    traj_tsne_metrics_louvain = extract_metrics(traj_tsne_eval_louvain, "t-SNE", "Trajectory", "Louvain")
    traj_umap_metrics_louvain = extract_metrics(traj_umap_eval_louvain, "UMAP", "Trajectory", "Louvain")
    
    # metrics for Leiden
    pbmc_pca_metrics_leiden = extract_metrics(pbmc_pca_eval_leiden, "PCA", "PBMC", "Leiden")
    pbmc_tsne_metrics_leiden = extract_metrics(pbmc_tsne_eval_leiden, "t-SNE", "PBMC", "Leiden")
    pbmc_umap_metrics_leiden = extract_metrics(pbmc_umap_eval_leiden, "UMAP", "PBMC", "Leiden")
    
    traj_pca_metrics_leiden = extract_metrics(traj_pca_eval_leiden, "PCA", "Trajectory", "Leiden")
    traj_tsne_metrics_leiden = extract_metrics(traj_tsne_eval_leiden, "t-SNE", "Trajectory", "Leiden")
    traj_umap_metrics_leiden = extract_metrics(traj_umap_eval_leiden, "UMAP", "Trajectory", "Leiden")
    
    # combine all
    all_metrics = (
        pbmc_pca_metrics_louvain + pbmc_tsne_metrics_louvain + pbmc_umap_metrics_louvain +
        traj_pca_metrics_louvain + traj_tsne_metrics_louvain + traj_umap_metrics_louvain +
        pbmc_pca_metrics_leiden + pbmc_tsne_metrics_leiden + pbmc_umap_metrics_leiden +
        traj_pca_metrics_leiden + traj_tsne_metrics_leiden + traj_umap_metrics_leiden
    )
    
    metrics_df = pd.DataFrame(all_metrics)
    
    return metrics_df

Visualize the comparison between Louvain and Leiden clustering!

In [ ]:
def plot_clustering_algorithm_comparison(performance_df):
    """performance metrics between Louvain and Leiden"""
    
    plt.figure(figsize=(18, 12))
    
    # ARI
    plt.subplot(2, 2, 1)
    sns.boxplot(x='Method', y='ARI', hue='Clustering_Algorithm', 
                data=performance_df, palette=['#3498db', '#e74c3c'])
    plt.title('Clustering Quality (ARI) by Algorithm')
    plt.tight_layout()
    
    # NMI
    plt.subplot(2, 2, 2)
    sns.boxplot(x='Method', y='NMI', hue='Clustering_Algorithm', 
                data=performance_df, palette=['#3498db', '#e74c3c'])
    plt.title('Clustering Quality (NMI) by Algorithm')
    plt.tight_layout()
    
    # dataset-specific comparison:PBMC
    plt.subplot(2, 2, 3)
    sns.boxplot(x='Method', y='ARI', hue='Clustering_Algorithm', 
                data=performance_df[performance_df['Dataset'] == 'PBMC'], 
                palette=['#3498db', '#e74c3c'])
    plt.title('PBMC Dataset - ARI by Clustering Algorithm')
    plt.tight_layout()
    
    # dataset-specific comparison: Trajectory
    plt.subplot(2, 2, 4)
    sns.boxplot(x='Method', y='ARI', hue='Clustering_Algorithm', 
                data=performance_df[performance_df['Dataset'] == 'Trajectory'], 
                palette=['#3498db', '#e74c3c'])
    plt.title('Trajectory Dataset - ARI by Clustering Algorithm')
    plt.tight_layout()
    
    plt.suptitle('Louvain vs Leiden Clustering Performance', fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

In [ ]:
# combined performance summary
performance_df_combined = create_performance_summary_with_clustering_comparison(
    # Louvain
    pbmc_pca_eval_louvain, pbmc_tsne_eval_louvain, pbmc_umap_eval_louvain,
    traj_pca_eval_louvain, traj_tsne_eval_louvain, traj_umap_eval_louvain,
    # Leiden
    pbmc_pca_eval_leiden, pbmc_tsne_eval_leiden, pbmc_umap_eval_leiden,
    traj_pca_eval_leiden, traj_tsne_eval_leiden, traj_umap_eval_leiden
)

print("Performance Summary with Clustering Algorithm Comparison:")
display(performance_df_combined)

plot_clustering_algorithm_comparison(performance_df_combined)

Step 11: Save Results and Create Environment Files

In [ ]:
# saving figures and results
os.makedirs('figures', exist_ok=True)
os.makedirs('results', exist_ok=True)

performance_df_combined.to_csv('results/performance_metrics.csv', index=False)

# environment.yml file
with open('environment.yml', 'w') as f:
    f.write("""name: scrna_dimred
channels:
  - conda-forge
  - bioconda
  - defaults
dependencies:
  - python=3.9
  - numpy
  - pandas
  - matplotlib
  - seaborn
  - scikit-learn
  - umap-learn
  - scanpy
  - anndata
  - jupyterlab
  - nb_conda_kernels
  - pip
  - pip:
    - leidenalg
""")

print("Results saved and environment.yml file created.")

Step 12: Generate a Summary and Conclusion

# Conclusions

## Summary of Findings

1. **PCA** 
   - Fastest runtime, but also suprisingly -somehow- best? clustering preservation
   - Good at preserving global structure
   - Best for initial dimensionality reduction before other methods

2. **t-SNE**
   - Slowest runtime, especially for larger datasets
   - Excellent at preserving local structure and clusters
   - Performance highly sensitive to perplexity parameter
   - Tends to fragment continuous trajectories ????

3. **UMAP**
   - Moderate runtime, faster than t-SNE
   - Good balance between local and global structure preservation
   - Best performance on the trajectory dataset
   - More stable across parameter configurations than t-SNE

## Optimal Parameters

- **PCA**: 50 components captured arround 30-40 variance
- **t-SNE**: Perplexity of 30 worked best across both datasets
- **UMAP**: n_neighbors=15, min_dist=0.1 provided the best balance between preserving clusters and trajectories

## Recommendations

- For exploratory analysis: Start with PCA, then UMAP
- For cluster visualization: UMAP with smaller min_dist
- For trajectory analysis: UMAP with larger n_neighbors
- For best computational efficiency: PCA followed by UMAP






plots show only ~30-35% variance explained with 100 components.

Revise clustering performance claims: PCA shows the highest clustering preservation, not the lowest as stated. This contradicts your conclusion but shows an interesting finding - PCA's linear projections maintain more of the original structure.

Parameter recommendations should be dataset-specific:

For PBMC: Higher n_neighbors (30-50) for UMAP
For trajectory: n_neighbors=15, min_dist=0.1 is indeed optimal
Explain PCA's strong performance: PCA works well for clustering despite simpler visualization - it preserved* global distances better.

Step 13: Generate a Presentation Script

# Presentation Script

## Introduction (3 min)
- Single-cell RNA-seq allows us to profile thousands of individual cells
- Dimensionality reduction is essential for visualization and analysis
- This project compares PCA, t-SNE, and UMAP on two distinct datasets

## Methods (5-7 min)
- Used Python (scanpy) for implementation
- Analyzed PBMC dataset (discrete cell types) and developmental trajectory
- Applied preprocessing pipeline: QC, normalization, feature selection
- Compared methods across multiple parameter settings
- Evaluated using: cluster preservation for both Louvain and Leiden (ARI/NMI), structure preservation (KNN), runtime

## Results - PBMC Dataset (2 min)
- PCA: Explained ~40% variance with 50-100 components
- t-SNE: Best separates distinct cell types, especially with perplexity=30
- UMAP: Balanced performance, faster than t-SNE
- Show visualizations and performance metrics

## Results - Trajectory Dataset (2 min)
- PCA: Captures linear aspects of differentiation, but limited in 2D
- t-SNE: Fragments continuous structures at low perplexity
- UMAP: Best preserves developmental trajectory, especially with higher n_neighbors
- Show visualizations and performance metrics

## Conclusions (2 min)
- Each method has strengths and weaknesses
- PCA is best for initial dimensionality reduction
- t-SNE excels at visualizing discrete clusters
- UMAP provides the best balance for most scRNA-seq applications
- Optimal workflow: PCA → UMAP with parameters tuned to analysis goals